In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [31]:

from FRVRS import nu, fu
from pandas import read_excel, to_datetime, Series
import numpy as np

In [3]:

# load data frames
data_frames_list = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df')
frvrs_logs_df = data_frames_list['frvrs_logs_df']
print(frvrs_logs_df.shape) # (829116, 125)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl.
(829116, 125)



# Create a DataFrame of all the File Stats

In [4]:

if nu.pickle_exists('file_stats_df'): file_stats_df = nu.load_object('file_stats_df')
else:
    
    # Get the columns that consistently have only one value in them per session
    single_value_cols_set = set(frvrs_logs_df.columns)
    for session_uuid, session_df in frvrs_logs_df.groupby('session_uuid'):
        single_value_cols = set([col for col in session_df.columns if session_df[col].nunique() == 1])
        single_value_cols_set = single_value_cols_set.intersection(single_value_cols)
    
    # Get the learner types from the spreadsheet
    if nu.pickle_exists('learner_types_df'): learner_types_df = nu.load_object('learner_types_df')
    else:
        file_path = '../data/xlsx/uuid_Master_with_Site_and_Learner_Type20240213.xlsx'
        learner_types_df = read_excel(file_path)
        learner_types_df.columns = ['session_file_date', 'session_file_name', 'session_uuid', 'responder_name', 'responder_type', 'site_name', 'encounter_layout']
        learner_types_df = learner_types_df.dropna(axis='columns', how='all')
        if ('session_file_date' in learner_types_df.columns):
            learner_types_df['session_file_date'] = to_datetime(learner_types_df['session_file_date'], infer_datetime_format=True)
        print(learner_types_df.shape) # (346, 6)
        nu.store_objects(learner_types_df=learner_types_df)
        nu.save_data_frames(learner_types_df=learner_types_df)
    
    file_stats_df = frvrs_logs_df[single_value_cols_set].merge(learner_types_df, how='left', on='session_uuid').drop_duplicates().reset_index(drop=True)
    for session_uuid, session_df in file_stats_df.groupby('session_uuid'):
        mask_series = session_df.session_file_date.isnull()
        if mask_series.any():
            session_file_date = fu.get_session_file_date(frvrs_logs_df, session_uuid)
            file_stats_df.loc[session_df[mask_series].index, 'session_file_date'] = session_file_date
    nu.store_objects(file_stats_df=file_stats_df)
    nu.save_data_frames(file_stats_df=file_stats_df)
print(file_stats_df.shape) # (429, 10)

(429, 11)


In [5]:

# Fix the null file dates
mask_series = file_stats_df.session_file_date.isnull()
if mask_series.any():
    print(f'I have {mask_series.sum()} scenes in my stats data frame without file dates.')
    for session_uuid, idx_df in file_stats_df[mask_series].groupby('session_uuid'):
        
        # Get the whole session history
        mask_series = (frvrs_logs_df.session_uuid == session_uuid)
        session_df = frvrs_logs_df[mask_series]
        
        session_file_date = session_df.event_time.min().date()
        file_stats_df.loc[idx_df.index, 'session_file_date'] = session_file_date
    nu.store_objects(file_stats_df=file_stats_df)
    nu.save_data_frames(file_stats_df=file_stats_df)
    mask_series = file_stats_df.session_file_date.isnull()
    print(f'I now have {mask_series.sum()} scenes in my stats data frame without file dates.')

In [6]:

file_stats_df.sample(8).T

,354,257,424,55,365,336,33,296
is_scene_aborted,False,False,False,False,False,False,False,False
logger_version,1.0,1.0,1.3,1.3,1.0,1.0,1.3,1.0
session_uuid,7de52180-d823-4f9e-9003-aeab8139970b,86366fd5-be94-43cc-a9e9-7c98609c0a66,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,f3cbb8a9-8d0c-48f0-ac0c-e379ac4d9411,95a83c35-3575-461a-9a53-8aa09c8afa62,2dd0d21c-1633-47de-848a-317df7ea3180,6b704e43-9ef4-4d5c-b39b-40c41648f8d3,a6b7ff70-3b20-48c6-86e8-744bad19f7d7
is_a_one_triage_file,False,True,True,True,False,False,True,True
file_name,v.1.0/clean-7de52180-d823-4f9e-9003-aeab813997...,Double runs removed/22.03.15.0926r.csv,v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv,All CSV files renamed by date/04.19.23.1013.csv,v.1.0/clean-95a83c35-3575-461a-9a53-8aa09c8afa...,v.1.0/clean-2dd0d21c-1633-47de-848a-317df7ea31...,All CSV files renamed by date/03.14.23.1554.csv,v.1.0/Clean a6b7ff70-3b20-48c6-86e8-744bad19f7...
session_file_date,2023-06-23 10:06:00,2022-03-15 00:00:00,2023-09-11 00:00:00,2023-04-19 10:13:00,2023-05-01 00:00:00,2023-06-23 00:00:00,2023-03-14 15:53:51,2023-05-12 00:00:00
responder_name,Tommy H,NaN,NaN,Bayden C,NaN,NaN,Heather W,NaN
responder_type,Paramedic,NaN,NaN,Paramedic,NaN,NaN,M4-ATEM,NaN
site_name,Franklin Twp,NaN,NaN,Madison Twp,NaN,NaN,OSUCOM,NaN
encounter_layout,First 11,NaN,NaN,First 11,NaN,NaN,First 11,NaN


In [7]:

# Label the military files
if 'scenario_environment' not in file_stats_df.columns:
    import re

    for session_uuid, idx_df in file_stats_df.groupby('session_uuid'):

        # Get the whole session history
        mask_series = (frvrs_logs_df.session_uuid == session_uuid)
        session_df = frvrs_logs_df[mask_series]

        if any(session_df.patient_id.dropna().map(lambda x: bool(re.search('(Military|Intelligence|Marine)', x, re.IGNORECASE)))):
            file_stats_df.loc[idx_df.index, 'scenario_environment'] = 'jungle'
    nu.store_objects(file_stats_df=file_stats_df)
    nu.save_data_frames(file_stats_df=file_stats_df)
mask_series = (file_stats_df.scenario_environment == 'jungle')
df = file_stats_df[mask_series].drop_duplicates()
print(df.shape) # (15, 11)
display(df.dropna(axis='columns', how='all'))

(15, 11)


,is_scene_aborted,logger_version,session_uuid,is_a_one_triage_file,file_name,session_file_date,scenario_environment
413,False,1.3,0b630a14-5acd-4e11-99df-8f59d804f807,False,v.1.3/0b630a14-5acd-4e11-99df-8f59d804f807.csv,2023-09-07,jungle
414,False,1.3,2a5106e4-984c-42a7-9edd-2fb3e6325d10,False,v.1.3/2a5106e4-984c-42a7-9edd-2fb3e6325d10.csv,2023-09-11,jungle
415,False,1.3,2faaa766-35d7-4fff-910c-f7b044bb913b,False,v.1.3/2faaa766-35d7-4fff-910c-f7b044bb913b.csv,2023-09-11,jungle
416,False,1.3,384462fc-969c-42cd-944f-726634faba4f,False,v.1.3/384462fc-969c-42cd-944f-726634faba4f.csv,2023-09-07,jungle
417,False,1.3,666ce61c-2ebc-40ee-902e-6f6aa42801ad,False,v.1.3/666ce61c-2ebc-40ee-902e-6f6aa42801ad.csv,2023-09-07,jungle
418,False,1.3,8b979d02-dbea-4d22-864b-7031425815cb,True,v.1.3/8b979d02-dbea-4d22-864b-7031425815cb.csv,2023-09-07,jungle
419,False,1.3,a3d6d913-7755-4e8d-a174-d5e491c4eac7,False,v.1.3/a3d6d913-7755-4e8d-a174-d5e491c4eac7.csv,2023-09-11,jungle
420,False,1.3,b06475e0-5ddd-4ff8-ba82-abf5331d6c9c,False,v.1.3/b06475e0-5ddd-4ff8-ba82-abf5331d6c9c.csv,2023-09-07,jungle
421,False,1.3,b0f6e371-e548-4e1d-adc1-92891a6ca6ca,True,v.1.3/b0f6e371-e548-4e1d-adc1-92891a6ca6ca.csv,2023-09-07,jungle
422,False,1.3,b11f14c2-8de9-4247-9753-1434cb392804,False,v.1.3/b11f14c2-8de9-4247-9753-1434cb392804.csv,2023-09-11,jungle


In [8]:

# Find out if any files dated the same are actually jungle
from datetime import date
from os import path as osp

mask_series = file_stats_df.session_file_date.isin([date(2023, 9, 7), date(2023, 9, 11)]) & (file_stats_df.scenario_environment != 'jungle')
df = file_stats_df[mask_series]
session_uuid = df.session_uuid.squeeze()
print(df.shape) # (1, 11)
file_name = df.file_name.squeeze()
print(osp.abspath(osp.join('../data/logs', file_name)))
mask_series = (frvrs_logs_df.session_uuid == session_uuid) & ~frvrs_logs_df.patient_id.isnull()
patients_list = sorted(frvrs_logs_df[mask_series].patient_id.unique())
session_file_date = df.session_file_date.squeeze()
print(
    f'Doug,\n\nThere is a file called {file_name} that was dated {session_file_date.date()} that only has {nu.conjunctify_nouns(patients_list)} in it - no military.'
    ' Was that session also in the jungle environment?'
)

(1, 11)
/mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/data/logs/v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv
Doug,

There is a file called v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv that was dated 2023-09-11 that only has Bob_1 Root, Bob_10 Root, Bob_12 Root, Bob_7 Root, Gary_16 Root, Gary_2 Root, Helga_14 Root, Helga_17 Root, Helga_6 Root, Helga_9 Root, Lily_0 Root, Lily_11 Root, Lily_15 Root, Lily_5 Root, Mike_13 Root, Mike_3 Root, Mike_4 Root, and Mike_8 Root in it - no military. Was that session also in the jungle environment?


In [9]:

for cn in file_stats_df.columns: print(cn, file_stats_df[cn].unique().shape[0])

is_scene_aborted 1
logger_version 2
session_uuid 426
is_a_one_triage_file 2
file_name 426
session_file_date 285
responder_name 235
responder_type 14
site_name 9
encounter_layout 3
scenario_environment 2


In [38]:

# Create a missing demographics Excel file, delete the jungle entries and the responder name column
# and then send it to Dave and Doug with Nick, me, and Ashu CC’d
missing_columns_list = ['responder_type', 'site_name', 'encounter_layout']
mask_series = False
for cn in missing_columns_list: mask_series |= file_stats_df[cn].isnull()
mask_series &= (file_stats_df.scenario_environment != 'jungle')
columns_list = []
for cn in file_stats_df.columns:
    if (file_stats_df[mask_series][cn].unique().shape[0] > 1) and (cn not in missing_columns_list): columns_list.append(cn)
df = file_stats_df[mask_series][columns_list+missing_columns_list].drop(columns=['responder_name'])
df.to_excel('../saves/xlsx/missing_demographics.xlsx', index=False)
stamps_list = [ts.timestamp() for ts in df.sort_values('session_file_date').session_file_date]

# It would also be good to include in the text of your email the row sum total and grouped by date range "out of" total by date:
missing_columns_str = []
for cn in missing_columns_list:
    mask_series = ~file_stats_df[cn].isnull() & (file_stats_df[cn] != 'Unknown')
    values_list = np.random.choice(file_stats_df[mask_series][cn].unique(), size=2, replace=False).tolist()
    missing_columns_str.append(f'{cn} (' + ', '.join(values_list) + ', etc.)')
missing_columns_str = nu.conjunctify_nouns(missing_columns_str)
print(
    f'Dave and Doug,\n\nWe need you to fill the missing (as of 2/15) data in the {missing_columns_str}'
    f' columns of the attached spreadsheet.\n\nThere are {df.shape[0]} rows of missing information:'
)

# 53 from 3/15-3/16/2022
# 30 from 11/30-12/6/2022
# 23 from 3/7-3/15/2023
# 14 from 4/19-4/25/2023
# 19 from 5/1-5/23/2023
# 9 from 6/1-6/23/2023
# 2 from 7/27/2023
# 9 from 8/1-8/16/2023
full_splits_list = nu.get_splits_list([ts.timestamp() for ts in file_stats_df.sort_values('session_file_date').session_file_date], value_difference=24*60*60)
for grouping in nu.get_splits_list(stamps_list, value_difference=24*60*60):
    from_date_obj = to_datetime(grouping[0], unit='s')
    from_date = from_date_obj.strftime('%m/%d')
    to_date_obj = to_datetime(grouping[-1], unit='s')
    to_date = to_date_obj.strftime('%m/%d/%Y')
    full_group = []
    for subgroup in full_splits_list:
        if grouping[0] in subgroup:
            full_group = subgroup
            break
    y = 'y'
    if full_group: y = len(full_group)
    date_range = f'{from_date} to {to_date}'
    if (from_date_obj == to_date_obj): date_range = from_date
    print(f'{len(grouping)} out of {y} from {date_range}')

Dave and Doug,

We need you to fill the missing (as of 2/15) data in the responder_type (EMT-I, TEMS, etc.), site_name (Union County, Madison Twp, etc.), and encounter_layout (DCEMS 11, First 11, etc.) columns of the attached spreadsheet.

There are 159 rows of missing information:
53 out of 53 from 03/15 to 03/16/2022
23 out of 23 from 11/30 to 12/02/2022
7 out of 7 from 12/06
10 out of 26 from 03/07 to 03/08/2023
12 out of 47 from 03/14 to 03/15/2023
3 out of 51 from 04/19 to 04/20/2023
11 out of 11 from 04/25
2 out of 2 from 05/01
7 out of 35 from 05/09 to 05/12/2023
8 out of 8 from 05/16
2 out of 2 from 05/22 to 05/23/2023
2 out of 11 from 06/08
2 out of 2 from 06/12
5 out of 11 from 06/23
2 out of 25 from 07/27
1 out of 26 from 08/01
4 out of 26 from 08/09 to 08/10/2023
4 out of 5 from 08/16
1 out of 6 from 09/11
